# Manual Labeling & Segmenting

## How to use this notebook:

1. Use the slider to move through frames.

2. Click Mark Start to mark the start of a shot.

3. Move slider to the end frame, click the shot label, then click Mark End.

4. Repeat for all shots in the video.

5. Click Save Labels to export all labeled segments as a CSV with frame numbers and corresponding seconds.

In [ ]:
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

video_file = '【nice angle】Kunlavut Vitidsarn vs Lu Guang Zu -Sin.mp4' # Adjust your video file here
video_path = f'../data/raw/{video_file}'  
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Data storage for labels: shot_id, start_frame, end_frame, label
labels = []
current_shot = {'start': None, 'end': None, 'label': None}
shot_counter = 1

# Widgets
image_widget = widgets.Image(format='jpeg', width=640, height=360)
frame_slider = widgets.IntSlider(min=0, max=total_frames - 1, step=1, description='Frame')
start_button = widgets.Button(description="Mark Start", button_style='success')
end_button = widgets.Button(description="Mark End", button_style='warning')
save_button = widgets.Button(description="Save Labels", button_style='info')
output = widgets.Output()

# Predefined shot labels
shot_labels = ["net shot", "lift", "clear", "smash", "drop", "drive", "block"]
label_buttons = []

# Create a button for each shot label
def on_label_button_click(label):
    current_shot['label'] = label
    with output:
        clear_output(wait=True)
        print(f"Selected label: {label}")

for label in shot_labels:
    button = widgets.Button(description=label, button_style='primary')
    button.on_click(lambda b, label=label: on_label_button_click(label))
    label_buttons.append(button)

def update_frame(change):
    frame_no = change['new']
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)
    ret, frame = cap.read()
    if ret:
        # Resize for display
        h, w = frame.shape[:2]
        max_w = 640
        scale = max_w / w
        frame = cv2.resize(frame, (max_w, int(h*scale)))
        _, jpeg = cv2.imencode('.jpg', frame)
        image_widget.value = jpeg.tobytes()

def mark_start(b):
    current_frame = frame_slider.value
    current_shot['start'] = current_frame
    with output:
        clear_output(wait=True)
        print(f"Start frame marked at {current_frame}")

def mark_end(b):
    current_frame = frame_slider.value
    if current_shot['start'] is None:
        with output:
            clear_output(wait=True)
            print("Please mark start frame first!")
        return
    if current_frame <= current_shot['start']:
        with output:
            clear_output(wait=True)
            print("End frame must be after start frame!")
        return
    if current_shot['label'] is None:
        with output:
            clear_output(wait=True)
            print("Please select a shot label!")
        return
    current_shot['end'] = current_frame
    global shot_counter
    labels.append({
        'shot_id': shot_counter,
        'start_frame': current_shot['start'],
        'end_frame': current_shot['end'],
        'label': current_shot['label']
    })
    with output:
        clear_output(wait=True)
        print(f"Saved shot {shot_counter}: {current_shot}")
    shot_counter += 1
    # Reset current shot
    current_shot['start'] = None
    current_shot['end'] = None
    current_shot['label'] = None
    # Reset label buttons
    for button in label_buttons:
        button.disabled = False

import os
def save_labels(b):
    if not labels:
        with output:
            clear_output(wait=True)
            print("No labels to save!")
        return
    
    # Create DataFrame from new labels
    df = pd.DataFrame(labels)
    df['start_sec'] = df['start_frame'] / fps
    df['end_sec'] = df['end_frame'] / fps

    # Path to save the CSV
    csv_path = '../data/output/manual_labels.csv'

    # Check if the CSV file already exists
    if os.path.exists(csv_path):
        # If it exists, load the existing data and append new data
        existing_df = pd.read_csv(csv_path)
        df = pd.concat([existing_df, df], ignore_index=True)
    
    # Save (either new or appended) data to the CSV
    df.to_csv(csv_path, index=False)
    
    with output:
        clear_output(wait=True)
        print(f"Labels saved to {csv_path}")
        display(df)


# Link widgets
frame_slider.observe(update_frame, names='value')
start_button.on_click(mark_start)
end_button.on_click(mark_end)
save_button.on_click(save_labels)

# Display interface
display(
    frame_slider,
    image_widget,
    start_button,
    widgets.HBox(label_buttons),  # Display the shot label buttons
    end_button, 
    save_button,
    output,
)

# Initialize first frame
update_frame({'new': 0})

IntSlider(value=0, description='Frame', max=21280)

Image(value=b'', format='jpeg', height='360', width='640')

Button(button_style='success', description='Mark Start', style=ButtonStyle())

Button(button_style='warning', description='Mark End', style=ButtonStyle())

Button(button_style='info', description='Save Labels', style=ButtonStyle())

Output()